<a href="https://colab.research.google.com/github/swarupmohalik/FFN/blob/main/MLApproxMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resources**

https://www.youtube.com/watch?v=3L1fRBLssAk

Convering keras to onnx

**Keras experiment**

Build a simple model, set weights and extract weights using Keras API's

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:


input_dim = 2
# Define the model
model = keras.Sequential([
    keras.layers.Dense(units=3, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(units=1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
model.summary()
model.layers[0].get_weights()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


[array([[-0.32896727, -0.04953778,  0.8804946 ],
        [ 0.49672043, -0.76843566, -0.18752927]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [4]:

l0_weights = np.array([[1,1,1], [2,2,2]])
l0_biases = np.array([10,10,10])
model.layers[0].set_weights([l0_weights, l0_biases])

l1_weights = np.array([[3],[3],[3]])
l1_biases = np.array([-1])
model.layers[1].set_weights([l1_weights, l1_biases])

In [5]:
model.layers[0].get_weights()

[array([[1., 1., 1.],
        [2., 2., 2.]], dtype=float32),
 array([10., 10., 10.], dtype=float32)]

In [6]:
model.layers[1].get_weights()

[array([[3.],
        [3.],
        [3.]], dtype=float32),
 array([-1.], dtype=float32)]

In [7]:
model.predict([[3,4]])

1/1 [==============================] - 0s 192ms/step


array([[188.]], dtype=float32)

In [8]:
# Get the list of layers in the model
layers = model.layers

# Loop through the layers and extract details
for layer in layers:
    layer_config = layer.get_config()
    layer_type = layer_config['name']
    units = layer_config['units']
    activation = layer_config['activation']

    print(f"Layer Type: {layer_type}")
    print(f"Number of Units: {units}")
    print(f"Activation Function: {activation}")
    print("=" * 40)

Layer Type: dense
Number of Units: 3
Activation Function: relu
Layer Type: dense_1
Number of Units: 1
Activation Function: linear


**Onnx model experiments**

Convert Keras model to onnx model, save
Load onnx model and get the details from the model.graph structure

In [9]:
!pip install tf2onnx onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26


In [10]:
import tf2onnx
import onnx
import onnxruntime

In [11]:
# Convert Keras model to ONNX model
onnx_model, _ = tf2onnx.convert.from_keras(model)

# Save the ONNX model to a file
onnx_model_path = 'your_model.onnx'
with open(onnx_model_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())

In [12]:
session = onnxruntime.InferenceSession('/content/your_model.onnx')
session.get_inputs()[0].shape

['unk__6', 2]

In [13]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
res = session.run([output_name], {input_name:[[3,4]]})
print(res)

[array([[188.]], dtype=float32)]


In [14]:
def raw_data_to_float(raw_data, data_type, shape):
    if data_type == onnx.TensorProto.FLOAT16:
        # Convert from half-precision (float16) to float32
        float_array = np.frombuffer(raw_data, dtype=np.float16)
        float_array = float_array.astype(np.float32)
    elif data_type == onnx.TensorProto.FLOAT:
        float_array = np.frombuffer(raw_data, dtype=np.float32)
    elif data_type == onnx.TensorProto.DOUBLE:
        float_array = np.frombuffer(raw_data, dtype=np.float64)
    else:
        raise ValueError(f"Unsupported data type: {data_type}")

    return float_array.reshape(shape)

In [15]:
# Access the graph of the ONNX model
graph = onnx_model.graph

# Loop through initializers (weights and biases) in the graph
for initializer in graph.initializer:
    print(f"Name: {initializer.name}")
    print(f"Shape: {initializer.dims}")
    float_data = raw_data_to_float(initializer.raw_data, initializer.data_type, initializer.dims)
    print(f"Values: {float_data}")  # or initializer.raw_data for bytes data
    print("=" * 40)

Name: sequential/dense_1/MatMul/ReadVariableOp:0
Shape: [3, 1]
Values: [[3.]
 [3.]
 [3.]]
Name: sequential/dense_1/BiasAdd/ReadVariableOp:0
Shape: [1]
Values: [-1.]
Name: sequential/dense/MatMul/ReadVariableOp:0
Shape: [2, 3]
Values: [[1. 1. 1.]
 [2. 2. 2.]]
Name: sequential/dense/BiasAdd/ReadVariableOp:0
Shape: [3]
Values: [10. 10. 10.]


In [16]:
graph

node {
  input: "dense_input"
  input: "sequential/dense/MatMul/ReadVariableOp:0"
  output: "sequential/dense/MatMul:0"
  name: "sequential/dense/MatMul"
  op_type: "MatMul"
}
node {
  input: "sequential/dense/MatMul:0"
  input: "sequential/dense/BiasAdd/ReadVariableOp:0"
  output: "sequential/dense/BiasAdd:0"
  name: "sequential/dense/BiasAdd"
  op_type: "Add"
}
node {
  input: "sequential/dense/BiasAdd:0"
  output: "sequential/dense/Relu:0"
  name: "sequential/dense/Relu"
  op_type: "Relu"
}
node {
  input: "sequential/dense/Relu:0"
  input: "sequential/dense_1/MatMul/ReadVariableOp:0"
  output: "sequential/dense_1/MatMul:0"
  name: "sequential/dense_1/MatMul"
  op_type: "MatMul"
}
node {
  input: "sequential/dense_1/MatMul:0"
  input: "sequential/dense_1/BiasAdd/ReadVariableOp:0"
  output: "dense_1"
  name: "sequential/dense_1/BiasAdd"
  op_type: "Add"
}
name: "tf2onnx"
initializer {
  dims: 3
  dims: 1
  data_type: 1
  name: "sequential/dense_1/MatMul/ReadVariableOp:0"
  raw_data: 